In [3]:
%run nc_underestimator-mutid.ipynb
import pyomo.environ as pyo

In [ ]:
import pyomo.environ as pyo

def build_pid_sp_model(
    T=50,                 # 时间步数
    h=0.1,                # 时间步长 Δt
    scenarios=None,       # 场景字典: {n: {"prob":..., "Ku":..., "tau":..., "d":[t->...], "sp":[t->...]}}
    weights=(1.0, 0.01),  # 目标权重 (w_e, w_u)
    bounds={"x":(-20,20), "u":(-20,20), "Kp":(0,100), "Ki":(0,1000), "Kd":(0,1000)},
    use_cvar=False,       # False=期望模型; True=CVaR
    alpha=0.95            # CVaR 的 1-α 尾部 (alpha 越大越保守)
):
    """
    受控对象：一阶线性系统（示例）
        tau(n) * x_dot + x = Ku(n)*u + d_n(t)
    PID：
        u = Kp*e + Ki*I + Kd*de/dt,   I_dot = e,   e = sp - x
    差分离散（隐式欧拉，稳定性更好）：
        x_{t+1} = x_t + (h/tau_n) * ( -x_{t+1} + Ku_n*u_{t+1} + d_{n,t+1} )
        I_{t+1} = I_t + h * e_{t+1}
        e_{t} = sp_{n,t} - x_{t}
    目标：
        EV:  Σ_n prob_n * Σ_t h * ( w_e * e_{n,t}^2 + w_u * u_{n,t}^2 )
        CVaR: min m + (1/(1-α)) Σ_n prob_n * z_n,  s.t. cost_n - m ≤ z_n, z_n ≥ 0
    """
    assert scenarios and len(scenarios) > 0, "请传入至少一个场景"
    Nset = sorted(scenarios.keys())
    prob = {n: scenarios[n]["prob"] for n in Nset}
    # 归一化概率（稳妥起见）
    s = sum(prob.values())
    for n in Nset:
        prob[n] /= s

    m = pyo.ConcreteModel()
    m.T = pyo.RangeSet(0, T)          # 包含 t=0 … T
    m.Tm = pyo.RangeSet(1, T)         # t=1 … T
    m.N = pyo.Set(initialize=Nset)

    # 共享的 PID 参数（非预见性约束通过“只建一次变量”天然实现）
    m.Kp = pyo.Var(bounds=bounds["Kp"])
    m.Ki = pyo.Var(bounds=bounds["Ki"])
    m.Kd = pyo.Var(bounds=bounds["Kd"])

    # 场景-时间变量
    m.x = pyo.Var(m.N, m.T, bounds=bounds["x"])
    m.u = pyo.Var(m.N, m.T, bounds=bounds["u"])
    m.e = pyo.Var(m.N, m.T)
    m.I = pyo.Var(m.N, m.T)

    # 便捷访问数据
    Ku = {n: scenarios[n]["Ku"] for n in Nset}
    tau = {n: scenarios[n]["tau"] for n in Nset}
    d = {n: scenarios[n]["d"] for n in Nset}     # 列表/数组，长度 T+1, 索引对齐 t=0..T
    sp = {n: scenarios[n]["sp"] for n in Nset}   # 同上

    # 误差定义：e_{n,t} = sp_{n,t} - x_{n,t}
    def _err_rule(m, n, t):
      return m.e[n, t] == sp[n][t] - m.x[n, t]
    m.err_def = pyo.Constraint(m.N, m.T, rule=_err_rule)

    # I 动态：I_{t} - I_{t-1} = h * e_t  (隐式欧拉)
    def _I_dyn(m, n, t):
      return m.I[n, t] == m.I[n, t-1] + h * m.e[n, t]
    m.I_dyn = pyo.Constraint(m.N, m.Tm, rule=_I_dyn)

    # 系统动态（隐式欧拉）：x_{t} = x_{t-1} + (h/tau) * ( -x_{t} + Ku*u_{t} + d_{t} )
    def _x_dyn(m, n, t):
      return m.x[n, t] == m.x[n, t-1] + (h / tau[n]) * ( - m.x[n, t] + Ku[n] * m.u[n, t] + d[n][t] )
    m.x_dyn = pyo.Constraint(m.N, m.Tm, rule=_x_dyn)

    # PID (差分版)：
    # 用“差分近似”代替导数项：de/dt ≈ (e_t - e_{t-1})/h ； t=0 时令导数为 0
    def _pid_rule(m, n, t):
      if t == 0:
        return m.u[n, t] == m.Kp * m.e[n, t] + m.Ki * m.I[n, t]  # 无 D 项
      return m.u[n, t] == m.Kp * m.e[n, t] + m.Ki * m.I[n, t] + m.Kd * (m.e[n, t] - m.e[n, t-1]) / h
    m.pid = pyo.Constraint(m.N, m.T, rule=_pid_rule)

    # 初值（可按需暴露为参数）
    x0 = 0.0
    I0 = 0.0
    def _x0_rule(m, n): return m.x[n, 0] == x0
    def _I0_rule(m, n): return m.I[n, 0] == I0
    m.x0c = pyo.Constraint(m.N, rule=_x0_rule)
    m.I0c = pyo.Constraint(m.N, rule=_I0_rule)

    # 每个场景的累计代价 cost_n
    w_e, w_u = weights
    m.cost = pyo.Var(m.N, domain=pyo.NonNegativeReals)
    def _cost_rule(m, n):
      return m.cost[n] == sum( h * ( w_e * m.e[n, t]**2 + w_u * m.u[n, t]**2 ) for t in m.T )
    m.cost_def = pyo.Constraint(m.N, rule=_cost_rule)

    if not use_cvar:
        # 期望值目标
        m.obj = pyo.Objective(expr = sum( prob[n] * m.cost[n] for n in m.N ), sense=pyo.minimize)
    else:
        # CVaR_α：min m + 1/(1-α) * Σ prob_n * z_n
        m.m = pyo.Var()  # VaR
        m.z = pyo.Var(m.N, domain=pyo.NonNegativeReals)
        m.cvar_con = pyo.Constraint(m.N, rule=lambda m,n: m.cost[n] - m.m <= m.z[n])
        m.obj = pyo.Objective(
            expr = m.m + (1.0/(1.0 - alpha)) * sum( prob[n] * m.z[n] for n in m.N ),
            sense=pyo.minimize
        )

    return m,  [m.Kp, m.Ki, m.Kd]

In [5]:
# ---------------------------
# 示例：构造三个场景的数据
if __name__ == "__main__":
    T = 60; h = 0.1
    times = [t for t in range(T+1)]

    def step_sp(t): return 1.0 if t*h >= 0.5 else 0.0
    def make_d(amp): return [amp*math.sin(0.5*t*h) for t in times]

    scenarios = {
        1: {"prob": 0.4, "Ku": 3.0, "tau": 2.0, "d": make_d(0.2), "sp": [step_sp(t) for t in times]},
        2: {"prob": 0.4, "Ku": 2.7, "tau": 1.8, "d": make_d(0.5), "sp": [step_sp(t) for t in times]},
        3: {"prob": 0.2, "Ku": 3.3, "tau": 2.2, "d": make_d(0.8), "sp": [step_sp(t) for t in times]},
    }

    m, _ = build_pid_sp_model(T=T, h=h, scenarios=scenarios, use_cvar=False, alpha=0.95)

    # 选一个合适的求解器：无整数变量 → NLP 推荐 IPOPT
    solver = pyo.SolverFactory("ipopt")
    # 可选的收敛稳健设置
    try:
        solver.options.update({
            "tol": 1e-6,
            "max_iter": 5000,
        })
    except Exception:
        pass

    res = solver.solve(m, tee=False)
    print(res.solver.status, res.solver.termination_condition)
    print("Kp, Ki, Kd =", pyo.value(m.Kp), pyo.value(m.Ki), pyo.value(m.Kd))
    ev = sum(pyo.value(m.cost[n]) * scenarios[n]["prob"] for n in m.N)
    print("Expected cost =", ev)

ok optimal
Kp, Ki, Kd = 19.385786860047425 5.852712758041871 0.0011278364030365099
Expected cost = 0.0397293376397264


In [ ]:
# ---------------------------
# 示例：构造一个场景的数据
if __name__ == "__main__":
    T = 60; h = 0.1
    times = [t for t in range(T+1)]

    def step_sp(t): return 1.0 if t*h >= 0.5 else 0.0
    def make_d(amp): return [amp*math.sin(0.5*t*h) for t in times]

    scenarios = {
        1: {"prob": 0.4, "Ku": 3.0, "tau": 2.0, "d": make_d(0.2), "sp": [step_sp(t) for t in times]},
    }
    scenarios = {
    1: {"prob": 0.4, "Ku": 3.0, "tau": 2.0, "d": make_d(0.2), "sp": [step_sp(t) for t in times]},
    2: {"prob": 0.4, "Ku": 2.7, "tau": 1.8, "d": make_d(0.5), "sp": [step_sp(t) for t in times]},
    3: {"prob": 0.2, "Ku": 3.3, "tau": 2.2, "d": make_d(0.8), "sp": [step_sp(t) for t in times]},
    }

    m, _ = build_pid_sp_model(T=T, h=h, scenarios=scenarios, use_cvar=False, alpha=0.95)

    # 选一个合适的求解器：无整数变量 → NLP 推荐 IPOPT
    solver = pyo.SolverFactory("ipopt")
    # 可选的收敛稳健设置
    try:
        solver.options.update({
            "tol": 1e-6,
            "max_iter": 5000,
        })
    except Exception:
        pass

    res = solver.solve(m, tee=False)
    print(res.solver.status, res.solver.termination_condition)
    print("Kp, Ki, Kd =", pyo.value(m.Kp), pyo.value(m.Ki), pyo.value(m.Kd))
    ev = sum(pyo.value(m.cost[n]) * scenarios[n]["prob"] for n in m.N)
    print("Expected cost =", ev)

ok optimal
Kp, Ki, Kd = 19.33370926948415 7.017332488357342 0.0017580675433941104
Expected cost = 0.016456331735819577


: 

In [7]:
import pyomo.environ as pyo

def build_pid_model(
    T=50,                 # 时间步数
    h=0.1,                # 步长 Δt
    scen=None,            # 单个场景数据: {"Ku":..., "tau":..., "d":[...], "sp":[...]}
    weights=(1.0, 0.01),  # 目标权重 (w_e, w_u)
    bounds={"x":(-20,20), "u":(-20,20), "Kp":(0,100), "Ki":(0,1000), "Kd":(0,1000)},
    use_cvar=False,       # 保留参数，但单场景下一般不需要
    alpha=0.95
):
    """
    单场景 PID 控制模型:
      tau * x_dot + x = Ku * u + d
      u = Kp*e + Ki*I + Kd*de/dt,   e = sp - x,   I_dot = e
    离散化: 隐式欧拉
    """
    assert scen is not None, "请提供一个场景字典"

    Ku, tau, d, sp = scen["Ku"], scen["tau"], scen["d"], scen["sp"]

    m = pyo.ConcreteModel()
    m.T = pyo.RangeSet(0, T)
    m.Tm = pyo.RangeSet(1, T)

    # 一阶段变量 (PID 参数)
    m.Kp = pyo.Var(bounds=bounds["Kp"])
    m.Ki = pyo.Var(bounds=bounds["Ki"])
    m.Kd = pyo.Var(bounds=bounds["Kd"])

    # 二阶段变量
    m.x = pyo.Var(m.T, bounds=bounds["x"])
    m.u = pyo.Var(m.T, bounds=bounds["u"])
    m.e = pyo.Var(m.T)
    m.I = pyo.Var(m.T)

    # 误差 e_t = sp_t - x_t
    def _err_rule(m, t): return m.e[t] == sp[t] - m.x[t]
    m.err_def = pyo.Constraint(m.T, rule=_err_rule)

    # I 动态
    def _I_dyn(m, t): return m.I[t] == m.I[t-1] + h*m.e[t]
    m.I_dyn = pyo.Constraint(m.Tm, rule=_I_dyn)

    # 系统动态 (隐式欧拉)
    def _x_dyn(m, t):
        return m.x[t] == m.x[t-1] + (h/tau)*(-m.x[t] + Ku*m.u[t] + d[t])
    m.x_dyn = pyo.Constraint(m.Tm, rule=_x_dyn)

    # PID 控制律
    def _pid_rule(m, t):
        if t == 0:
            return m.u[t] == m.Kp*m.e[t] + m.Ki*m.I[t]
        return m.u[t] == m.Kp*m.e[t] + m.Ki*m.I[t] + m.Kd*(m.e[t]-m.e[t-1])/h
    m.pid = pyo.Constraint(m.T, rule=_pid_rule)

    # 初值
    m.x0 = pyo.Constraint(expr=m.x[0] == 0)
    m.I0 = pyo.Constraint(expr=m.I[0] == 0)

    # 成本函数 (单场景就是一个值)
    w_e, w_u = weights
    m.cost = pyo.Expression(expr=sum(
        h*(w_e*m.e[t]**2 + w_u*m.u[t]**2) for t in m.T
    ))

    # 目标: 单场景情况下就直接最小化 cost
    #m.obj = pyo.Objective(expr=m.cost, sense=pyo.minimize)
    m.obj_expr = pyo.Expression(expr = m.cost)

    return m,  [m.Kp, m.Ki, m.Kd]


In [8]:
T, h = 60, 0.1
times = [t for t in range(T+1)]

def step_sp(t): return 1.0 if t*h >= 0.5 else 0.0
def make_d(amp): return [amp*math.sin(0.5*t*h) for t in times]

scenarios = {
    1: {"prob": 0.4, "Ku": 3.0, "tau": 2.0, "d": make_d(0.2), "sp": [step_sp(t) for t in times]},
    2: {"prob": 0.4, "Ku": 2.7, "tau": 1.8, "d": make_d(0.5), "sp": [step_sp(t) for t in times]},
    3: {"prob": 0.2, "Ku": 3.3, "tau": 2.2, "d": make_d(0.8), "sp": [step_sp(t) for t in times]},
}
'''scen = {
    "Ku": 3.0,
    "tau": 2.0,
    "d": make_d(0.3),
    "sp": [step_sp(t) for t in times],
}'''
model_list = []
first_stg_vars_list = []
for _, scen in scenarios.items():
    m, m_f_stg_vars = build_pid_model(T=T, h=h, scen=scen)
    model_list.append(m)
    first_stg_vars_list.append(m_f_stg_vars)
bounds={"x":(-20,20), "u":(-20,20), "Kp":(0,100), "Ki":(0,1000), "Kd":(0,1000)}
m_tmpl = pyo.ConcreteModel()
m_tmpl.T = pyo.RangeSet(0, T)
m_tmpl.Tm = pyo.RangeSet(1, T)
m_tmpl.Kp = pyo.Var(bounds=bounds["Kp"])
m_tmpl.Ki = pyo.Var(bounds=bounds["Ki"])
m_tmpl.Kd = pyo.Var(bounds=bounds["Kd"])
m_tmpl_list = [m_tmpl, [m_tmpl.Kp, m_tmpl.Ki, m_tmpl.Kd]]


nc_underest(model_list, first_stg_vars_list, m_tmpl_list, target_nodes=5,
                picture_shown=False, v_list=False, tolerance=1e-8
)

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
# ---------------------------
# 示例：构造三个场景的数据
if __name__ == "__main__":
    T = 60; h = 0.1
    times = [t for t in range(T+1)]

    def step_sp(t): return 1.0 if t*h >= 0.5 else 0.0
    def make_d(amp): return [amp*math.sin(0.5*t*h) for t in times]

    scenarios = {
        1: {"prob": 0.4, "Ku": 3.0, "tau": 2.0, "d": make_d(0.2), "sp": [step_sp(t) for t in times]},
        2: {"prob": 0.4, "Ku": 2.7, "tau": 1.8, "d": make_d(0.5), "sp": [step_sp(t) for t in times]},
        3: {"prob": 0.2, "Ku": 3.3, "tau": 2.2, "d": make_d(0.8), "sp": [step_sp(t) for t in times]},
    }

    m, _ = build_pid_sp_model(T=T, h=h, scenarios=scenarios, use_cvar=False, alpha=0.95)

    # 选一个合适的求解器：无整数变量 → NLP 推荐 IPOPT
    solver = pyo.SolverFactory("ipopt")
    # 可选的收敛稳健设置
    try:
        solver.options.update({
            "tol": 1e-6,
            "max_iter": 5000,
        })
    except Exception:
        pass

    res = solver.solve(m, tee=False)
    print(res.solver.status, res.solver.termination_condition)
    print("Kp, Ki, Kd =", pyo.value(m.Kp), pyo.value(m.Ki), pyo.value(m.Kd))
    ev = sum(pyo.value(m.cost[n]) * scenarios[n]["prob"] for n in m.N)
    print("Expected cost =", ev)